<a href="https://colab.research.google.com/github/nandiniii2404/topsis_pretrained/blob/main/topsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Step 1: Define the data
data = {
    'Model': ['BART', 'T5', 'PEGASUS', 'DistilBART', 'LED'],
    'ROUGE-1': [0.44, 0.42, 0.45, 0.41, 0.43],
    'ROUGE-2': [0.21, 0.20, 0.22, 0.19, 0.20],
    'Inference Time': [0.90, 0.85, 1.00, 0.65, 1.20],
    'Model Size': [1600, 900, 2000, 1350, 1100]
}

In [3]:
df = pd.DataFrame(data)

In [4]:
# Step 2: Normalize the decision matrix
criteria = ['ROUGE-1', 'ROUGE-2', 'Inference Time', 'Model Size']
weights = np.array([0.3, 0.3, 0.2, 0.2])
impacts = ['+', '+', '-', '-']

norm_df = df.copy()
for col in criteria:
    norm = np.sqrt(np.sum(df[col] ** 2))
    norm_df[col] = df[col] / norm

In [5]:
# Step 3: Apply weights
for i, col in enumerate(criteria):
    norm_df[col] = norm_df[col] * weights[i]

In [6]:
# Step 4: Identify ideal best and worst
ideal_best = []
ideal_worst = []

for i, col in enumerate(criteria):
    if impacts[i] == '+':
        ideal_best.append(norm_df[col].max())
        ideal_worst.append(norm_df[col].min())
    else:
        ideal_best.append(norm_df[col].min())
        ideal_worst.append(norm_df[col].max())

In [7]:
# Step 5: Calculate Euclidean distances
norm_df['S+'] = np.sqrt(np.sum((norm_df[criteria] - ideal_best) ** 2, axis=1))
norm_df['S-'] = np.sqrt(np.sum((norm_df[criteria] - ideal_worst) ** 2, axis=1))


In [8]:
# Step 6: Calculate TOPSIS scores
norm_df['Topsis Score'] = norm_df['S-'] / (norm_df['S+'] + norm_df['S-'])
norm_df['Rank'] = norm_df['Topsis Score'].rank(ascending=False).astype(int)


In [9]:
# Step 7: Final output
final_result = norm_df[['Model', 'Topsis Score', 'Rank']].sort_values(by='Rank')
print(final_result.to_string(index=False))

     Model  Topsis Score  Rank
        T5      0.753316     1
DistilBART      0.645309     2
       LED      0.503098     3
      BART      0.451246     4
   PEGASUS      0.284059     5
